In [1]:
import pandas as pd
import sqlalchemy.sql
from db import engine , engine_core

In [3]:
sales_query = """
SELECT 
	o.order_id,
	customer_id,
	order_status,
	order_purchase_timestamp,
	order_approved_at,
	order_delivered_carrier_date,
	order_delivered_customer_date,
	order_estimated_delivery_date,
	i.product_id,
	i.seller_id,
	price,
	freight_value 
FROM 
	orders o
LEFT JOIN items i
	ON o.order_id  = i.order_id;
"""
sales_df = pd.read_sql(sales_query,engine)
sales_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_id,seller_id,price,freight_value
0,00061f2a7bc09da83e415a52dc8a4af1,c6fc061d86fab1e2b2eac259bac71a49,delivered,2018-03-24 22:16:10,2018-03-24 22:28:09,2018-03-27 05:26:55,2018-03-29 00:04:19,2018-04-09 00:00:00,d63c1011f49d98b976c352955b1c4bea,cc419e0650a3c5ba77189a1882b7556a,59.99,8.88
1,0009c9a17f916a706d71784483a5d643,8a250edc40ebc5c3940ebc940f16a7eb,delivered,2018-04-25 09:10:41,2018-04-25 09:33:19,2018-04-27 15:14:00,2018-04-30 17:54:25,2018-05-09 00:00:00,3f27ac8e699df3d300ec4a5d8c5cf0b2,fcb5ace8bcc92f75707dc0f01a27d269,639.00,11.34
2,000e906b789b55f64edcb1f84030f90d,6a3b2fc9f270df258605e22bef19fd88,delivered,2017-11-21 18:54:23,2017-11-21 19:09:02,2017-11-22 20:46:54,2017-12-09 17:27:23,2017-12-07 00:00:00,57d79905de06d8897872c551bfd09358,ea8482cd71df3c1969d7b9473ff13abc,21.99,11.85
3,000f25f4d72195062c040b12dce9a18a,86ade8ebc14f18dd6c3ea75bd0eb7534,delivered,2018-03-07 10:33:13,2018-03-07 11:10:11,2018-03-16 15:58:31,2018-03-22 16:51:52,2018-04-11 00:00:00,1c05e0964302b6cf68ca0d15f326c6ba,7c67e1448b00f6e969d365cea6b010ab,119.99,44.40
4,0014ae671de39511f7575066200733b7,41065d9dcea52218c3943d2eed072b97,delivered,2017-05-22 13:49:03,2017-05-23 03:15:24,2017-05-29 09:04:02,2017-06-07 13:52:52,2017-06-13 00:00:00,23365beed316535b4105bd800c46670e,92eb0f42c21942b6552362b9b114707d,16.50,14.10


In [4]:
## getting null values count so we can transform them into usable data for our visualisation
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113425 entries, 0 to 113424
Data columns (total 12 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       113425 non-null  object 
 1   customer_id                    113425 non-null  object 
 2   order_status                   113425 non-null  object 
 3   order_purchase_timestamp       113425 non-null  object 
 4   order_approved_at              113264 non-null  object 
 5   order_delivered_carrier_date   111457 non-null  object 
 6   order_delivered_customer_date  110196 non-null  object 
 7   order_estimated_delivery_date  113425 non-null  object 
 8   product_id                     112650 non-null  object 
 9   seller_id                      112650 non-null  object 
 10  price                          112650 non-null  float64
 11  freight_value                  112650 non-null  float64
dtypes: float64(2), object(10)
memo

In [3]:
sales_df.isnull().any() # Returns True if the column contains any NaN

order_id                         False
customer_id                      False
order_status                     False
order_purchase_timestamp         False
order_approved_at                 True
order_delivered_carrier_date      True
order_delivered_customer_date     True
order_estimated_delivery_date    False
product_id                        True
seller_id                         True
price                             True
freight_value                     True
dtype: bool

In [4]:
sales_df.loc[sales_df["product_id"].isnull(), "order_status"].unique()

array(['unavailable', 'canceled', 'created', 'shipped', 'invoiced'],
      dtype=object)

In [5]:
sales_df['order_status'].value_counts()

order_status
delivered      110197
shipped          1186
canceled          706
unavailable       610
invoiced          361
processing        357
created             5
approved            3
Name: count, dtype: int64

In [5]:
sales_df["product_id"] = sales_df["product_id"].fillna("999")
sales_df["seller_id"] = sales_df["seller_id"].fillna("999")
sales_df['order_approved_at'] = sales_df['order_approved_at'].fillna("19900101")
sales_df['order_delivered_carrier_date'] = sales_df['order_delivered_carrier_date'].fillna("19900101")
sales_df['order_delivered_customer_date'] = sales_df['order_delivered_customer_date'].fillna("19900101")

In [5]:
sales_df.isnull().any()

order_id                         False
customer_id                      False
order_status                     False
order_purchase_timestamp         False
order_approved_at                False
order_delivered_carrier_date     False
order_delivered_customer_date    False
order_estimated_delivery_date    False
product_id                       False
seller_id                        False
price                             True
freight_value                     True
dtype: bool

In [6]:
sales_df['order_delivered_customer_date'].head()

0    2018-03-29 00:04:19
1    2018-04-30 17:54:25
2    2017-12-09 17:27:23
3    2018-03-22 16:51:52
4    2017-06-07 13:52:52
Name: order_delivered_customer_date, dtype: object

In [12]:
sales_df['order_delivered_customer_date'].dtypes

dtype('O')

In [6]:
sales_df['order_delivered_customer_date'] = pd.to_datetime(sales_df['order_delivered_customer_date'], errors='coerce')
sales_df['order_estimated_delivery_date'] = pd.to_datetime(sales_df['order_estimated_delivery_date'], errors='coerce')
sales_df['order_delivered_carrier_date'] = pd.to_datetime(sales_df['order_delivered_carrier_date'], errors='coerce')
sales_df['order_purchase_timestamp'] = pd.to_datetime(sales_df['order_purchase_timestamp'], errors='coerce')

In [7]:
sales_df['order_delivered_customer_date'].dtypes

dtype('<M8[ns]')

In [8]:
sales_df[sales_df.duplicated()]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_id,seller_id,price,freight_value
7,001d8f0e34a38c37f7dba2a37d4eba8b,d987da9fb4086ab7c2c0f83963cd6722,delivered,2017-05-14 17:19:44,2017-05-14 17:35:11,2017-05-24 15:45:01,2017-05-26 13:14:50,2017-05-24,e67307ff0f15ade43fcb6e670be7a74c,f4aba7c0bca51484c30ab7bdc34bcdd1,18.99,7.78
45,00f6ac4816d12f49026b09f2aa77d15f,9bb70fa63970b9d88191a90481d551da,delivered,2018-08-06 17:17:40,2018-08-06 17:30:14,2018-08-07 14:39:00,2018-08-13 16:18:48,2018-08-27,b532349fe46b38fbc7bb3914c1bdae07,1025f0e2d44d7041d6cf58b6550e0bfa,35.00,19.44
46,00f6ac4816d12f49026b09f2aa77d15f,9bb70fa63970b9d88191a90481d551da,delivered,2018-08-06 17:17:40,2018-08-06 17:30:14,2018-08-07 14:39:00,2018-08-13 16:18:48,2018-08-27,b532349fe46b38fbc7bb3914c1bdae07,1025f0e2d44d7041d6cf58b6550e0bfa,35.00,19.44
55,01115c06246220def210911556bdc0e5,7db5bde0f0fcee8817a4c317cf05429a,delivered,2018-03-14 09:20:38,2018-03-14 09:35:34,2018-03-14 19:22:05,2018-04-03 00:07:46,2018-04-04,c31539851232e3accbf0ef4c571f97f2,602044f2c16190c2c6e45eb35c2e21cb,52.50,19.34
71,014e36af18c280a94d880a16b2f00713,0523cd19f27a70bda29f2efb68a2db10,delivered,2017-11-01 18:14:32,2017-11-02 02:35:56,2017-11-03 19:14:58,2017-11-09 20:18:46,2017-11-23,33f789f4b4fa74053af0199658b78853,897060da8b9a21f655304d50fd935913,30.00,12.69
...,...,...,...,...,...,...,...,...,...,...,...,...
113264,ff4935343715e7a6dae3e550f73a6459,8bbd68396f2112f979111bfdd363e286,delivered,2017-10-10 15:41:10,2017-10-12 02:29:16,2017-10-16 20:57:40,2017-10-23 18:43:06,2017-11-13,013e6676e0e3529e5909ff54370daddf,1c68394e931a64f90ea236c5ea590300,299.90,16.85
113278,ff9770f7299db492e5fe01b4a231341b,c57b89d756843557ab53f7c679ea0572,delivered,2018-03-12 07:41:09,2018-03-12 07:50:28,2018-03-16 18:06:53,2018-04-04 15:05:24,2018-04-06,67bd616e1ba0d3d3e8545f3113b0140d,e9779976487b77c6d4ac45f75ec7afe9,16.49,18.23
113286,ffb18bf111fa70edf316eb0390427986,5c8f393ae866c2a2d2e775dbc2409a59,delivered,2017-11-27 13:29:05,2017-11-27 13:39:22,2017-11-28 22:15:05,2017-12-05 18:38:53,2017-12-21,599dc392f7a23273471b068d72408224,a08692680c77d30a0b4280da5df01c5a,235.00,19.30
113289,ffb9a9cd00c74c11c24aa30b3d78e03b,81a0150b8f472d0e5e82a7da198b93bc,delivered,2017-03-11 17:20:21,2017-03-11 17:20:21,2017-03-22 08:48:30,2017-03-28 15:26:02,2017-04-13,fec565c4e3ad965c73fb1a21bb809257,da8622b14eb17ae2831f4ac5b9dab84a,89.90,18.34


In [9]:
sales_df.drop_duplicates(inplace=True) ## supprimer les doublons definitivement

In [10]:
sales_df.describe() 

,order_purchase_timestamp,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,price,freight_value
count,103200,101398,100195,103200,102425.000000,102425.000000
mean,2017-12-31 09:37:41.742626048,2018-01-04 22:11:21.820223488,2018-01-14 11:17:50.464214784,2018-01-24 05:06:07.534883840,124.422006,20.111152
min,2016-09-04 21:15:19,2016-10-08 10:34:01,2016-10-11 13:46:32,2016-09-30 00:00:00,0.850000,0.000000
25%,2017-09-12 16:00:29.249999872,2017-09-15 23:20:56.249999872,2017-09-25 21:59:43.500000,2017-10-03 00:00:00,40.000000,13.140000
50%,2018-01-18 19:47:54.500000,2018-01-24 11:08:32,2018-02-02 16:04:10,2018-02-15 00:00:00,78.800000,16.320000
75%,2018-05-04 17:01:38.750000128,2018-05-08 13:51:00,2018-05-15 23:27:31,2018-05-28 00:00:00,139.000000,21.220000
max,2018-10-17 17:30:18,2018-09-11 19:48:28,2018-10-17 13:22:46,2018-11-12 00:00:00,6735.000000,409.680000
std,NaN,NaN,NaN,NaN,188.926084,15.884700


In [11]:
## en reversera maintenant notre data dans notre data warehouse
sales_df.to_sql('sales_fact',engine_core)

200

In [12]:
### dimensions
## in pandas you can read data via queries or just give the table name
cust = pd.read_sql('customers',engine)
cust.head()

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [13]:
cust.describe()

,index,customer_zip_code_prefix
count,99441.000000,99441.000000
mean,49720.000000,35137.474583
std,28706.288396,29797.938996
min,0.000000,1003.000000
25%,24860.000000,11347.000000
50%,49720.000000,24416.000000
75%,74580.000000,58900.000000
max,99440.000000,99990.000000


In [14]:
cust[cust.duplicated()]

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state


In [16]:
cust.isnull().any()
## i need now to use the index (surrogate keys) and map them in my facts instead of using the whole key
## to methodes i can make it happen here in pandas or use an update query
## but for now i'll be using sql

index                       False
customer_id                 False
customer_unique_id          False
customer_zip_code_prefix    False
customer_city               False
customer_state              False
dtype: bool

In [17]:
cust['index'] = cust['index'] + 1 
cust.head()

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,1,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,2,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,3,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,4,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,5,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [18]:
cust.to_sql('dim_customers',engine_core)

441

In [2]:
prod = pd.read_sql('products',engine)
prod.head()

,index,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [3]:
prod[prod.duplicated()]

,index,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm


In [4]:
prod.isnull().any()

index                         False
product_id                    False
product_category_name          True
product_name_lenght            True
product_description_lenght     True
product_photos_qty             True
product_weight_g               True
product_length_cm              True
product_height_cm              True
product_width_cm               True
dtype: bool

In [5]:
prod = prod.drop(columns=['product_name_lenght','product_description_lenght','product_photos_qty','product_weight_g','product_width_cm','product_height_cm','product_length_cm'])
prod.head()

,index,product_id,product_category_name
0,0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria
1,1,3aa071139cb16b67ca9e5dea641aaa2f,artes
2,2,96bd76ec8810374ed1b65e291975717f,esporte_lazer
3,3,cef67bcfe19066a932b7673e239eb23d,bebes
4,4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas


In [6]:
prod['product_category_name'] = prod['product_category_name'].fillna('NoCategory')

In [7]:
prod.isnull().any()

index                    False
product_id               False
product_category_name    False
dtype: bool

In [8]:
prod.head()

,index,product_id,product_category_name
0,0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria
1,1,3aa071139cb16b67ca9e5dea641aaa2f,artes
2,2,96bd76ec8810374ed1b65e291975717f,esporte_lazer
3,3,cef67bcfe19066a932b7673e239eb23d,bebes
4,4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas


In [9]:
prod.to_sql('dim_products',engine_core)

951

In [12]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   index                  32951 non-null  int64 
 1   product_id             32951 non-null  object
 2   product_category_name  32951 non-null  object
dtypes: int64(1), object(2)
memory usage: 772.4+ KB
